In [1]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import nltk

In [2]:
df = pd.read_csv('../MasterProject/combined_data.csv')
df.head()

,sentiment,Text
0,sadness,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,happy,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [3]:
for i,s in enumerate(df['sentiment']):
    if s == 'fear':
        df.loc[i, 'sentiment'] = 0
    if s == 'happy':
        df.loc[i, 'sentiment'] = 1
    if s == 'sadness':
        df.loc[i, 'sentiment'] = 2
    if s == 'neutral':
        df.loc[i, 'sentiment'] = 3
    if s == 'love':
        df.loc[i, 'sentiment'] = 4
    if s == 'anger':
        df.loc[i, 'sentiment'] = 5
    if s == 'surprise':
        df.loc[i, 'sentiment'] = 6
    if s == 'relief':
        df.loc[i, 'sentiment'] = 7
    

In [15]:
category_names = ['fear', 'happy', 'sadness', 'neutral', 'love', 'anger', 'surprise', 'relief']

In [4]:
df['sentiment'].value_counts()

0    16241
1    13508
2     9796
3     8960
4     4720
5     4069
6     2639
7     1526
Name: sentiment, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Text'], 
                                                    df['sentiment'],test_size=0.20, 
                                                    random_state=1)
y_train = y_train.astype('int')
y_test = y_train.astype('int')


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words = 'english')
X_train_counts = count_vect.fit_transform(X_train)

In [8]:
print(X_train_counts)

  (0, 14627)	1
  (0, 15399)	1
  (0, 8725)	1
  (0, 16554)	1
  (0, 18061)	1
  (0, 9205)	1
  (1, 7766)	1
  (1, 9969)	1
  (1, 32812)	1
  (1, 26743)	1
  (1, 8481)	1
  (2, 20746)	1
  (2, 15709)	1
  (2, 30103)	1
  (2, 20260)	1
  (2, 15478)	1
  (2, 33800)	1
  (2, 25983)	1
  (3, 43237)	1
  (3, 32820)	1
  (3, 15705)	1
  (3, 47738)	1
  (3, 25676)	1
  (3, 27805)	1
  (3, 40877)	1
  :	:
  (49163, 30106)	1
  (49164, 17968)	1
  (49164, 33802)	1
  (49164, 47590)	1
  (49164, 26283)	1
  (49164, 47343)	1
  (49164, 2758)	1
  (49164, 21704)	1
  (49165, 29581)	1
  (49165, 26136)	1
  (49165, 18722)	1
  (49165, 8670)	1
  (49165, 17964)	1
  (49165, 43522)	1
  (49165, 2309)	1
  (49165, 43415)	1
  (49165, 26310)	1
  (49166, 11420)	1
  (49166, 18963)	1
  (49166, 17933)	1
  (49166, 29104)	1
  (49166, 42883)	1
  (49166, 10072)	1
  (49166, 17983)	1
  (49166, 42187)	1


In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False)
X_train_tf = tf_transformer.fit_transform(X_train_counts)
print(X_train_tf)

  (0, 8725)	0.4082482904638631
  (0, 9205)	0.4082482904638631
  (0, 14627)	0.4082482904638631
  (0, 15399)	0.4082482904638631
  (0, 16554)	0.4082482904638631
  (0, 18061)	0.4082482904638631
  (1, 7766)	0.4472135954999579
  (1, 8481)	0.4472135954999579
  (1, 9969)	0.4472135954999579
  (1, 26743)	0.4472135954999579
  (1, 32812)	0.4472135954999579
  (2, 15478)	0.3779644730092272
  (2, 15709)	0.3779644730092272
  (2, 20260)	0.3779644730092272
  (2, 20746)	0.3779644730092272
  (2, 25983)	0.3779644730092272
  (2, 30103)	0.3779644730092272
  (2, 33800)	0.3779644730092272
  (3, 15705)	0.3779644730092272
  (3, 25676)	0.3779644730092272
  (3, 27805)	0.3779644730092272
  (3, 32820)	0.3779644730092272
  (3, 40877)	0.3779644730092272
  (3, 43237)	0.3779644730092272
  (3, 47738)	0.3779644730092272
  :	:
  (49163, 31422)	0.3779644730092272
  (49164, 2758)	0.3779644730092272
  (49164, 17968)	0.3779644730092272
  (49164, 21704)	0.3779644730092272
  (49164, 26283)	0.3779644730092272
  (49164, 33802)	0.3

In [10]:
clf = MLPClassifier(random_state=1, max_iter=2).fit(X_train_tf, y_train)

/home/owner/.local/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [13]:
X_test_counts = count_vect.transform(X_test)
X_test_tf = tf_transformer.transform(X_test_counts)

In [14]:
predicted = clf.predict(X_test_tf)

In [17]:
for text, category in zip(X_test, predicted):
    print('%r => %s' % (text, category_names[category]))

'@blackmantra yes indeed!  nice to see you on twitter! Why do you ask? xx' => neutral
'@aami1982 probably a bit short on the rude words and I bet it was on time' => fear
'just finished tlking to laura 2 hours and 7 minutes lol... going to sleep. night ppl' => neutral
'@LOCALDUDER I appreciate the effort. My skin is much softer than your screen' => happy
'@Eier_Von_Abe  I have no idea!! but all white?? weird!! but not bad' => fear
'@marinesetwas dann mach mal!  #12seconds' => neutral
'i was also given several shiny presents because my friends are really rather cool i actually prefer late birthday presents to early ones as it extends the period of feeling beloved significant segments of all and sundry and is more unexpected' => happy
'@Bellisma can we please?' => neutral
'i feel very glad as in facebook we have many cricket related pages which are providing news views score update of cricket and website like cricnepal cricket' => fear
'i suppose that is how a lot of things feel when you 

'Now i cant find him  Gonna be a sleepless night tonight' => happy
'@LilMissOpinion I just noticed that  They are so ridiculous' => fear
'Just came home from my sister where I spent the night' => happy
'@DrRus You too, Rus!' => neutral
"Everyone's talking about the goodies at #BEA09.  I wanna be there." => neutral
'i feel the presence of the divine with you when you are buried inside me smiling down at me your sweat dripping into my eager mouth' => happy
'i then asked as i often do in these situations how i could fix this so she wouldnt feel like i hated her because of my lack of postings on her facebook page' => fear
'@LolzyluvsJB Neither could i! And i looked on ebay and it was like 250 euro for two!!!! but i doubt my parents would fork that amount out' => fear
"Sick......and no it's not the swine flu atleast I hope not" => fear
"had 2 leave the mrs @sweetbullshit cuz i have to go 2 work  really didn't want to." => fear
"I gave up on tumblr because the API isn't pointing to the priva

'@gfalcone601 salt and vinegar, cheese and onion make your breathe smell lol  xx' => happy
'this goes out to the one n only smirker   cheers dave!  aka jak  aka the best chest ever! ? http://blip.fm/~5jjcc' => fear
'@willyemai Indeed!' => neutral
'@wimnms well i miss her i think she would really like it  re:trina' => sadness
'Hmmm.... My whole body feels sore!' => fear
'i feel rich indeed' => happy
'i feel shaky discussing it with anybody especially in public as though i m a little ball of explosive tears just waiting to spill out everywhere' => fear
'@BleedBlacknGold meee.. I have a Penguin wallpaper on the backround on my phone, computer, ps3, ipod touch and on my twitter' => neutral
'i didnt feel i had put in half the effort or time and well quite frankly didnt feel like the pressure of it all' => fear
'@kepp been there done that. It always ended up with me selling the machine and going back to my mac' => neutral
'i feel unbearably tortured knowing that im helpless i cant invade nor

'Hanging with kevin tomorrow!  out goes being productive.' => happy
'i was feeling particularly beaten up by istanbul and homesickish i passed a burger king and the door opened and the smell hit me full in the face and suddenly i was in snowpea my white nissan stanza in the drive thru of the burger king on rt' => fear
'no more arrested development. i am sad.' => sadness
'http://twitpic.com/4jh4l - I picked out a watch !! Yey' => happy
"Apparently it's 21 degrees in Kilkenny now! And some comedy would be a great remedy to my boredom! Why the hell did I leave Kilkenny!?" => neutral
'took 8 trips to basement to move books from my room - i have a floor again' => fear
'im feeling a bit lonely without comments to respond to c' => sadness
"Gonna go and see my sister at Mount Sinai ....  she's there for at least another 24 hours." => neutral
'i do remember the feel of the book and being incredibly impressed with it and knowing that ill have to read it again' => fear
'ill admit to feeling a lit